# Bloomberg BQuant Webinar Series: Turkish Lira Crisis
This is a companion notebook to the [Bloomberg Turkish Lira Crisis](https://blinks.bloomberg.com/screens/PLYR%20VOD%20330281983) webinar.

In [ ]:
import bql
import pandas as pd
import bqviz as bqv
from collections import OrderedDict
from bqplot import *

In [ ]:
bq = bql.Service()

A bql request needs two parts

A "get()" -> This contains the fields that you want to retrieve

A "for()" -> This contains the universe that I want to request data for

In [ ]:
bql_str = """get(px_last)for('XU100 Index')"""

resp_str = bq.execute(bql_str)

resp_str[0].df()

In [ ]:
fld = bq.data.px_last()
tckr = 'XU100 Index'

req_obj = bql.Request(tckr,fld)

req_obj.to_string()

In [ ]:
resp_obj = bq.execute(req_obj)

resp_obj[0].df()

In [ ]:
tkr_lst = ['XU100 Index','USDTRY BGN CURNCY','USDTRY3M BGNL CURNCY']

prc = bq.data.px_last(start = '-15y',frq='w')
pct_chg = bq.func.pct_diff(prc)
fld = {'Pct Chg' : pct_chg}

req = bql.Request(tkr_lst,fld)
resp = bq.execute(req)

resp[0].df().tail()

In [ ]:
ts_df = resp[0].df()
ts_df = ts_df.reset_index()
ts_df = ts_df.pivot(index='DATE',columns='ID')['Pct Chg']

ts_df.tail()

In [ ]:
bqv.InteractiveLinePlot(ts_df).show()

In [ ]:
index = bq.univ.members('XU100 Index')

memb_prc = bq.data.px_last(start='2018-01-01',end='2018-08-14',currency='TRY',ca_adj='FULL')
memb_prc = bq.func.dropna(memb_prc)

index_flds =OrderedDict([('YTD',bq.func.pct_chg(memb_prc)),
            ('Mkt Cap',bq.data.cur_mkt_cap(currency='USD'))])


req_index = bql.Request(index,index_flds)
resp_index = bq.execute(req_index)

ytd_df = resp_index[0].df().reset_index()
ytd_df.tail()

In [ ]:
bqv.HistPlot(ytd_df[['YTD']],bins=100).show()

In [ ]:
index = bq.univ.members('XU100 Index')
mem_bonds = bq.univ.bonds(index,issuedby='ENTITY')

name = bq.data.id()["'orig_ids:0'"]
debt_cur = bq.func.if_(bq.data.CRNCY() == 'TRY', 'TRY', 'Non TRY')

grouping = [name,debt_cur]

amt = bq.data.amt_outstanding(currency='USD')/1000000

total_amt = {'Amt (USD)':bq.func.sum(bq.func.group(amt,grouping))}

req_debt= bql.Request(mem_bonds,total_amt)
resp_debt = bq.execute(req_debt)

resp_debt[0].df().tail()

In [ ]:
debt_df = resp_debt[0].df()

debt_df.rename(columns={"ID().'ORIG_IDS:0'":'ID',"IF((CRNCY()=='TRY'),'TRY','NON TRY')":'Type'},inplace=True)

debt_df = debt_df.pivot(index='ID', columns='Type',values = 'Amt (USD)')
debt_df = debt_df.fillna(0)
debt_df = debt_df.reset_index()
debt_df.tail()

In [ ]:
mkt_df = resp_index[1].df().reset_index()

all_df = pd.merge(debt_df,ytd_df,how='left', on=['ID'])
all_df = pd.merge(all_df,mkt_df,how='left', on=['ID'])
all_df.tail()

In [ ]:
sc_x = LinearScale()
sc_y = LinearScale()
c_sc = ColorScale()

scatter = Scatter(x = all_df['Non TRY'],
                 y = all_df['Mkt Cap'],
                 color = all_df['YTD'],
                 names = all_df['ID'],
                  display_names=False,
                 scales={'x': sc_x, 'y': sc_y, 'color':c_sc },
                 tooltip=Tooltip(fields=['name']))

ax_x = Axis(label='Non Try Debt',scale=sc_x)
ax_y = Axis(label='Market Cap',scale=sc_y,orientation='vertical')
ax_c = ColorAxis(label='YTD Rtn',scale=c_sc,side='right')

Figure(axes=[ax_x,ax_y,ax_c], marks=[scatter])